In [1]:
rm(list = ls())
setwd(rprojroot::find_rstudio_root_file())
# source("base/Preprocess_data.R")
# source("base/Regression_model.R")
# source("base/Knn_model.R")
# source("base/Export_data.R")
# source("base/Scores.R")


In [2]:
### Forecasts
# input data

run_model <- function(datetime_initialisation,forecast_mode,export,...) {
    source("base/Preprocess_data.R")
    source("base/Regression_model.R")
    source("base/Knn_model.R")
    source("base/Export_data.R")
    source("base/Scores.R")
  #catchment_code <- "5410002"
  #datetime_initialisation = lubridate::make_date(2016,6,1)
  #horizon = horizon_mode(window_method = "dynamic", month_start = 9, month_end = 3)
  #predictor_list <- c("tem_mean_6months")
  #remove_wys <- c(1990,1940,2013)
  #water_units = waterunits(q = "m^3/s", y = "GL")
  #forecast_mode <- "both"
  #data_location_paths = get_default_datasets_path(meteo = "ens30avg", hydro = "ERA5Ens_SKGE")

    #pre-processed data
    data_input = preprocess_data(
        datetime_initialisation,
        forecast_mode,...)
    # ensemble volume forecast
    data_fore = forecast_vol_ensemble(
      data_input = data_input,
      forecast_mode = forecast_mode)
    # # ensemble flow forecast
    #q_fore = run_q_forecast(
    #  data_input = data_input,
    #  data_fore = data_fore,
    #  forecast_mode = forecast_mode)
    #export results
    output = export_data(
      data_input = data_input,
      data_fore = data_fore,
      export = export)
  
  return(output)
  
}

In [3]:
#all available catchments
catchments_attributes_filename = "data_input/attributes/attributes_49catchments_ChileCentral.csv" 
cod_cuencas = read.csv(catchments_attributes_filename)$cod_cuenca
cod_cuencas

[1] 3414001 3421001 3430003 3820001 4302001 4311001 4313001 4320001 4501001
[10] 4503001 4511002 4513001 4515002 4522002 4531002 4703002 5100001 5101001
[19] 5200001 5401003 5410002 5411001 5414001 5707002 5710001 5721001 5722001
[28] 5722002 5730008 5741001 6003001 6008005 6013001 6027001 6028001 7103001
[37] 7104002 7112001 7115001 7317005 7321002 7330001 7350003 7354002 7355002
[46] 7358001 7381001 8104001 8106001

In [4]:
#months of initialisation
months_initialisation = seq(4,9)
months_initialisation

[1] 4 5 6 7 8 9

In [5]:
# test climate indices
# climate indices
climate_indices = c("MEIv2", "SOI", "ONI", "BIENSO")
months_horizon = seq(1,2,1)
climate_grid = expand.grid(climate_indices, months_horizon)
climate_predictors = paste(climate_grid[,1], "mean", paste0(climate_grid[,2],"months"), sep = "_")
climate_predictors


[1] "MEIv2_mean_1months"  "SOI_mean_1months"    "ONI_mean_1months"   
[4] "BIENSO_mean_1months" "MEIv2_mean_2months"  "SOI_mean_2months"   
[7] "ONI_mean_2months"    "BIENSO_mean_2months"

In [6]:
cod_cuencas

[1] 3414001 3421001 3430003 3820001 4302001 4311001 4313001 4320001 4501001
[10] 4503001 4511002 4513001 4515002 4522002 4531002 4703002 5100001 5101001
[19] 5200001 5401003 5410002 5411001 5414001 5707002 5710001 5721001 5722001
[28] 5722002 5730008 5741001 6003001 6008005 6013001 6027001 6028001 7103001
[37] 7104002 7112001 7115001 7317005 7321002 7330001 7350003 7354002 7355002
[46] 7358001 7381001 8104001 8106001

In [7]:
run_model(     
    catchment_code = sample(cod_cuencas,1),
    datetime_initialisation = lubridate::make_date(2016,sample(months_initialisation,1),1),
    horizon = horizon_mode(window_method = "static", month_start = 9, month_end = 3),
    predictor_list = sample(climate_predictors,1),
    remove_wys = seq(1950,1979),
    water_units = waterunits(q = "m^3/s", y = "GL"),
    forecast_mode = "cv",
    data_location_paths = get_default_datasets_path(meteo = "ens30avg", hydro = "ERA5Ens_SKGE"),
    export = 'scores')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: lattice

Loading required package: latticeExtra

Loading required package: polynom

Loading required package: reshape


Attaching package: ‘reshape’


The following object is masked from ‘package:lubridate’:

    stamp


The following object is masked from ‘package:

rmse_det,r2_det,mae_obs,crps_ens,crpss_climatology
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
112.6628,0.01160456,89.61796,63.80108,0.2880771


In [20]:
library(foreach)
#library(doParallel)
#registerDoParallel(cores = parallel::detectCores())
library(doSNOW)
library(dplyr)
cl <- makeCluster(4)
registerDoSNOW(cl)

list_lengths <- function(...){
  list_lengths <- lapply(list(...), length)
  total_iter <- prod(unlist(list_lengths))
  return(total_iter)
}

p1 = months_initialisation #sample(months_initialisation,2)
p2 = climate_predictors #sample(climate_predictors,2)
p3 = cod_cuencas #sample(cod_cuencas,2)


  

In [21]:
max_iter = list_lengths(p1,p2,p3)
pb <- txtProgressBar(max = max_iter, style = 3)
opts = list(progress = function(n) setTxtProgressBar(pb, n))
          
model <-
  foreach(month_initialisation= p1,.combine = "c") %:%
  foreach(climate_predictor = p2,.combine = "c") %:%
  foreach(catchment_code=p3,.options.snow=opts) %dopar% {    
      
    run_model(     
    catchment_code = catchment_code,
    datetime_initialisation = lubridate::make_date(2022,sample(months_initialisation,1),1),
    horizon = horizon_mode(window_method = "static", month_start = 9, month_end = 3),
    predictor_list = climate_predictor,
    remove_wys = c(1950,1979),
    water_units = waterunits(q = "m^3/s", y = "GL"),
    forecast_mode = "cv",
    data_location_paths = get_default_datasets_path(meteo = NULL, hydro = NULL),
    export = 'scores')
      
  }
model = purrr::transpose(model)

close(pb)
stopCluster(cl)
#stopImplicitCluster()


  |======================================================================| 100%


In [22]:
rbindlist(model$info[1])

catchment_code,datetime_initialisation,predictor_list,horizon,data_location_paths,water_units,forecast_mode,remove_wys,predictor_list_corrected
<int>,<date>,<chr>,<list>,<list>,<list>,<chr>,<list>,<chr>
3414001,2022-09-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months


In [23]:
rbindlist(model$scores)

rmse_det,r2_det,mae_obs,crps_ens,crpss_climatology
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
23.593135,0.0128208132,17.619892,13.113321,0.25576607
13.493073,0.0009689215,7.051830,6.432764,0.08778798
23.931352,0.0839673426,19.328404,13.175939,0.31831209
174.478318,0.0207409309,98.546479,87.510403,0.11198853
5.871791,0.2606793617,4.407703,3.188106,0.27669674
37.071117,0.1095984379,21.593582,19.697395,0.08781251
66.112404,0.0211669524,43.858067,34.993977,0.20210854
249.704869,0.0000605365,153.682344,127.395326,0.17104774
57.502356,0.1875483667,42.324519,30.488455,0.27965030


In [24]:
# stopImplicitCluster()
df <- cbind(rbindlist(model$info),rbindlist(model$scores))
# modify names
df <- dplyr::rename(df,'predictor_name' = 'predictor_list_corrected')

# order columns
#df$month_initialisation <- factor(df$month_initialisation,levels = months_initialisation)
df$predictor_name <- factor(df$predictor_name)
df$catchment_code <- as.numeric(df$catchment_code)
df


catchment_code,datetime_initialisation,predictor_list,horizon,data_location_paths,water_units,forecast_mode,remove_wys,predictor_name,rmse_det,r2_det,mae_obs,crps_ens,crpss_climatology
<dbl>,<date>,<chr>,<list>,<list>,<list>,<chr>,<list>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3414001,2022-09-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,23.593135,0.0128208132,17.619892,13.113321,0.25576607
3421001,2022-07-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,13.493073,0.0009689215,7.051830,6.432764,0.08778798
3430003,2022-06-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,23.931352,0.0839673426,19.328404,13.175939,0.31831209
3820001,2022-04-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,174.478318,0.0207409309,98.546479,87.510403,0.11198853
4302001,2022-08-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,5.871791,0.2606793617,4.407703,3.188106,0.27669674
4311001,2022-04-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,37.071117,0.1095984379,21.593582,19.697395,0.08781251
4313001,2022-06-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,66.112404,0.0211669524,43.858067,34.993977,0.20210854
4320001,2022-05-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,249.704869,0.0000605365,153.682344,127.395326,0.17104774
4501001,2022-07-01,MEIv2_mean_1months,"static, 9 , 3","data_input/attributes/attributes_49catchments_ChileCentral.csv , data_input/flows/flows_mm_monthly_49catchments_ChileCentral.csv , data_input/climate_index_variables/indices_mensuales_1979_present.csv","m^3/s, GL",cv,"1950, 1979",MEIv2_mean_1months,57.502356,0.1875483667,42.324519,30.488455,0.27965030


In [25]:
saveRDS(df,paste0("data_output/scores/RDS/model_results_",today(),".RDS"))
